# Aims to merge initial, partiel, flux (new + evt)

In [3]:
import boto3, json
import dask.dataframe as dd
import pandas as pd
import Match_inpi_insee.aws_connectors as aws
#from tqdm.notebook import tqdm
#import tqdm
%load_ext autoreload
%autoreload 2

In [4]:
inpi_nature = 'PP' #* nature → ACTES/COMPTES/ETS/etc

import_path = 'SIRETISATION'
match_type='matche'
import_full_path =  "{}/{}".format(import_path,match_type)

initial_file_name = "{}_{}_{}_{}.{}".format('insee',inpi_nature,'initial',match_type,'gz')#insee_PP_initial_matche.gz
new_file_name = "{}_{}_{}_{}.{}".format('insee',inpi_nature,'new',match_type,'gz')#insee_PP_new_matche.gz
partiel_file_name = "{}_{}_{}_{}.{}".format('insee',inpi_nature,'partiel',match_type,'gz')
initial_filepath = "{}/{}/{}".format(import_path,match_type,initial_file_name)
new_filepath = "{}/{}/{}".format(import_path,match_type,new_file_name)
partiel_filepath = "{}/{}/{}".format(import_path,match_type,partiel_file_name)

In [5]:
# instanciate AWS connection
instance_aws = 'https://calfdata.s3.eu-west-3.amazonaws.com'
bucket = 'calfdata'
AWS_connection = aws.aws_instantiate(instance_aws, bucket)
#AWS_connection.uploadFileBucket(pathfile =inpi_import_json_filepath)

In [6]:
# get files

pp_initial = AWS_connection.url_instance_bucket(path_file = initial_filepath)
pp_new = AWS_connection.url_instance_bucket(path_file = new_filepath)
pp_partiel = AWS_connection.url_instance_bucket(path_file = partiel_filepath)

In [7]:
## INPI
dtypes__ = {'Siren':'object',
            'Adresse_Ligne1': 'object',
       'Code_Commune': 'object',
       'Code_Postal': 'object',
       'Conjoint_Collab_Pseudo': 'object',
       'DAP_Adresse_Ligne1': 'object',
       'DAP_Code_Commune': 'object',
       'Date_Immatriculation': 'object',
       'Date_1re_Immatriculation': 'object',
       'Date_Radiation': 'object',
       'Date_Greffe': 'object',
       'Sans_Activité': 'object'}

data_pp_new= dd.read_csv(pp_new,
                         compression='gzip',
                         dtype=dtypes__,
                         blocksize=None,
                         low_memory=False
                      )
data_pp_new['source']='new'
data_pp_new.head()

,siren,siret,statutDiffusionEtablissement,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Sans_Activité,Adresse_Ligne1,Code_Postal,Code_Commune,DAP_Adresse_Ligne1,DAP_Code_Commune,Conjoint_Collab_Pseudo,Date_Greffe,_merge,url,source
0,314488321,31448832100014,O,2017-12-18,NaN,NaN,NaN,NaN,34290,34009,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/314488321,new
1,314488321,31448832100022,O,2017-12-18,NaN,NaN,NaN,NaN,34290,34009,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/314488321,new
2,314488321,31448832100030,O,2017-12-18,NaN,NaN,NaN,NaN,34290,34009,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/314488321,new
3,338065873,33806587300014,O,2017-12-29,NaN,NaN,NaN,NaN,31820,31417,NaN,NaN,NaN,2017-12-29,both,https://data.inpi.fr/entreprises/338065873,new
4,338065873,33806587300022,O,2017-12-29,NaN,NaN,NaN,NaN,31820,31417,NaN,NaN,NaN,2017-12-29,both,https://data.inpi.fr/entreprises/338065873,new


In [8]:
data_pp_initial= dd.read_csv(pp_initial,
                         compression='gzip',
                         dtype=dtypes__,
                         blocksize=None,
                         low_memory=False
                      )
data_pp_initial['source']='initial'
data_pp_initial.head()

,siren,siret,statutDiffusionEtablissement,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Sans_Activité,Adresse_Ligne1,Code_Postal,Code_Commune,DAP_Adresse_Ligne1,DAP_Code_Commune,Conjoint_Collab_Pseudo,Date_Greffe,_merge,url,source
0,5741509,574150900011,O,1957-06-01,NaN,NaN,NaN,NaN,04270,NaN,NaN,NaN,NaN,2009-01-01,both,https://data.inpi.fr/entreprises/005741509,initial
1,5915533,591553300011,O,1959-12-28,NaN,NaN,NaN,NaN,80220,80373,NaN,NaN,NaN,2009-01-01,both,https://data.inpi.fr/entreprises/005915533,initial
2,6472419,647241900032,O,1964-05-27,NaN,NaN,NaN,9 rue de Conan,44780,44098,NaN,NaN,NaN,2017-05-04,both,https://data.inpi.fr/entreprises/006472419,initial
3,6641104,664110400010,O,1966-07-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-06-06,both,https://data.inpi.fr/entreprises/006641104,initial
4,7241532,724153200014,O,1980-06-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-04,both,https://data.inpi.fr/entreprises/007241532,initial


In [9]:
data_pp_partiel= dd.read_csv(pp_partiel,
                         compression='gzip',
                         dtype=dtypes__,
                         blocksize=None,
                         low_memory=False
                      )
data_pp_partiel['source']='partiel'
data_pp_partiel.head()

,siren,siret,statutDiffusionEtablissement,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Sans_Activité,Adresse_Ligne1,Code_Postal,Code_Commune,DAP_Adresse_Ligne1,DAP_Code_Commune,Conjoint_Collab_Pseudo,Date_Greffe,_merge,url,source
0,36410462,3641046200018,O,1964-02-05,NaN,NaN,NaN,NaN,06600,06004,NaN,NaN,NaN,2017-11-17,both,https://data.inpi.fr/entreprises/036410462,partiel
1,308810662,30881066200012,O,1976-12-10,NaN,NaN,NaN,NaN,65200,NaN,NaN,NaN,NaN,2017-05-05,both,https://data.inpi.fr/entreprises/308810662,partiel
2,309682284,30968228400018,O,1981-03-18,NaN,NaN,NaN,NaN,84410,84017,NaN,NaN,NaN,2018-01-26,both,https://data.inpi.fr/entreprises/309682284,partiel
3,310059316,31005931600011,O,1977-05-17,NaN,NaN,NaN,NaN,13500,13056,NaN,NaN,NaN,2017-05-16,both,https://data.inpi.fr/entreprises/310059316,partiel
4,311124929,31112492900028,O,1983-11-30,NaN,NaN,NaN,NaN,13100,13001,NaN,NaN,NaN,2017-06-02,both,https://data.inpi.fr/entreprises/311124929,partiel


In [10]:
# Union all data
data_pp_all=data_pp_new.append(data_pp_initial)
data_pp_all=data_pp_all.append(data_pp_partiel)
data_pp_all.head()

,siren,siret,statutDiffusionEtablissement,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Sans_Activité,Adresse_Ligne1,Code_Postal,Code_Commune,DAP_Adresse_Ligne1,DAP_Code_Commune,Conjoint_Collab_Pseudo,Date_Greffe,_merge,url,source
0,314488321,31448832100014,O,2017-12-18,NaN,NaN,NaN,NaN,34290,34009,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/314488321,new
1,314488321,31448832100022,O,2017-12-18,NaN,NaN,NaN,NaN,34290,34009,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/314488321,new
2,314488321,31448832100030,O,2017-12-18,NaN,NaN,NaN,NaN,34290,34009,NaN,NaN,NaN,2017-12-18,both,https://data.inpi.fr/entreprises/314488321,new
3,338065873,33806587300014,O,2017-12-29,NaN,NaN,NaN,NaN,31820,31417,NaN,NaN,NaN,2017-12-29,both,https://data.inpi.fr/entreprises/338065873,new
4,338065873,33806587300022,O,2017-12-29,NaN,NaN,NaN,NaN,31820,31417,NaN,NaN,NaN,2017-12-29,both,https://data.inpi.fr/entreprises/338065873,new


In [11]:
# check source repartition
data_pp_all['source'].value_counts().compute()

initial    1189644
partiel      25987
new          19862
Name: source, dtype: int64

In [12]:
# sort by siren
data_pp_all_sorted = data_pp_all.compute().sort_values(['siren','Date_Greffe','source'])

In [13]:
# define save_to paths
inpi_save_to_root_path = 'SIRETISATION/matche'
gz_filename = 'insee_PP_all_matche.gz'
save_to_full_filename =  "{}/{}".format(inpi_save_to_root_path,gz_filename)
save_to_full_filename


'SIRETISATION/matche/insee_PP_all_matche.gz'

In [15]:
gz_output = data_pp_all_sorted.to_csv(
    gz_filename,
    index = False,
    compression='gzip')

AWS_connection.save_to_s3(file_name=gz_filename,file_path=save_to_full_filename)

In [56]:
def siren_unique(df):
    """
    """
    print("Nombre total obs: {}".format(len(df)))
    count_ = (df
              .groupby('siren')['siren']
              .count()
              .rename('count')
              .reset_index()
              .groupby('count')['count']
              .count()
              .reset_index(name='total_count')
              .set_index('count')
              # .compute()
              .assign(pct=lambda x: x/x.sum())
              .iloc[:10, :]
              .style
              .format('{:,.2%}', subset=['pct'])
              )
    return count_

In [104]:
# count nb siret by siren
siren_unique(data_pp_all_sorted)

Nombre total obs: 1235493


,total_count,pct
count,,
1,434502,60.06%
2,175301,24.23%
3,64689,8.94%
4,27837,3.85%
5,8788,1.21%
6,6444,0.89%
7,1325,0.18%
8,2046,0.28%
9,443,0.06%


In [106]:
# count siren that are in several sources
count_sources = (data_pp_all_sorted.groupby(['siren'])['source']
              .nunique()
              .rename('count')
 .reset_index()
              .groupby('count')['count']
              .count()
)
count_sources

count
1    719150
2      4323
3         4
Name: count, dtype: int64

In [96]:
# get siren that come from 3 different sources
(data_pp_all_sorted.groupby(['siren'])['source']
              .nunique()
              .rename('count')
 .loc[lambda x: x ==3] #.loc[lambda x: x> 1]
)

siren
344981501    3
349599092    3
502042732    3
829212828    3
Name: count, dtype: int64

In [97]:
# view data about one siren that is in 3 different sources
data_pp_all_sorted.loc[lambda x: x['siren'].isin(['344981501'])]

,siren,siret,statutDiffusionEtablissement,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Sans_Activité,Adresse_Ligne1,Code_Postal,Code_Commune,DAP_Adresse_Ligne1,DAP_Code_Commune,Conjoint_Collab_Pseudo,Date_Greffe,_merge,url,source
618499,344981501,34498150100028,O,1993-02-23,NaN,NaN,NaN,NaN,20137,NaN,NaN,NaN,NaN,2003-09-02,both,https://data.inpi.fr/entreprises/344981501,initial
618500,344981501,34498150100036,O,1993-02-23,NaN,NaN,NaN,NaN,20137,NaN,NaN,NaN,NaN,2003-09-02,both,https://data.inpi.fr/entreprises/344981501,initial
618501,344981501,34498150100044,O,1993-02-23,NaN,NaN,NaN,NaN,20137,NaN,NaN,NaN,NaN,2003-09-02,both,https://data.inpi.fr/entreprises/344981501,initial
618502,344981501,34498150100051,O,1993-02-23,NaN,NaN,NaN,NaN,20137,NaN,NaN,NaN,NaN,2003-09-02,both,https://data.inpi.fr/entreprises/344981501,initial
13371,344981501,34498150100028,O,1993-02-23,NaN,NaN,NaN,NaN,20137,NaN,NaN,NaN,NaN,2017-09-14,both,https://data.inpi.fr/entreprises/344981501,partiel
13372,344981501,34498150100036,O,1993-02-23,NaN,NaN,NaN,NaN,20137,NaN,NaN,NaN,NaN,2017-09-14,both,https://data.inpi.fr/entreprises/344981501,partiel
13373,344981501,34498150100044,O,1993-02-23,NaN,NaN,NaN,NaN,20137,NaN,NaN,NaN,NaN,2017-09-14,both,https://data.inpi.fr/entreprises/344981501,partiel
13374,344981501,34498150100051,O,1993-02-23,NaN,NaN,NaN,NaN,20137,NaN,NaN,NaN,NaN,2017-09-14,both,https://data.inpi.fr/entreprises/344981501,partiel
10587,344981501,34498150100028,O,2017-12-21,NaN,NaN,NaN,NaN,20137,2A247,NaN,NaN,NaN,2017-12-21,both,https://data.inpi.fr/entreprises/344981501,new
10588,344981501,34498150100036,O,2017-12-21,NaN,NaN,NaN,NaN,20137,2A247,NaN,NaN,NaN,2017-12-21,both,https://data.inpi.fr/entreprises/344981501,new


In [107]:
# get siren that come from 2 different sources
(data_pp_all_sorted.groupby(['siren'])['source']
              .nunique()
              .rename('count')
 .loc[lambda x: x ==2]
)

siren
6041099      2
6840540      2
45712452     2
46410726     2
46412375     2
46710810     2
46811527     2
46812046     2
46910147     2
47011192     2
47011465     2
47311352     2
56721467     2
70702691     2
85612273     2
86911633     2
86912417     2
87212106     2
95877775     2
97271423     2
300043981    2
300207354    2
300432515    2
300546454    2
300613247    2
300639960    2
300944915    2
301105615    2
301208716    2
301260105    2
            ..
854401809    2
860406925    2
868400078    2
868404112    2
871402905    2
872405287    2
877140558    2
877240051    2
887210573    2
896772068    2
917340663    2
926741216    2
927340570    2
936040682    2
936340074    2
950051326    2
950325233    2
950328948    2
950374371    2
950389676    2
950411165    2
950428557    2
950463885    2
950573303    2
950606095    2
950611178    2
950627323    2
968404137    2
969401835    2
977111020    2
Name: count, Length: 4323, dtype: int64

In [90]:
# view data about one siren that is in 2 different sources
data_pp_all_sorted.loc[lambda x: x['siren'].isin(['6041099'])]

,siren,siret,statutDiffusionEtablissement,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Sans_Activité,Adresse_Ligne1,Code_Postal,Code_Commune,DAP_Adresse_Ligne1,DAP_Code_Commune,Conjoint_Collab_Pseudo,Date_Greffe,_merge,url,source
277903,6041099,604109900018,O,1960-05-24,NaN,NaN,NaN,NaN,04120,NaN,NaN,NaN,NaN,2009-01-01,both,https://data.inpi.fr/entreprises/006041099,initial
277904,6041099,604109900026,O,1960-05-24,NaN,NaN,NaN,NaN,04120,NaN,NaN,NaN,NaN,2009-01-01,both,https://data.inpi.fr/entreprises/006041099,initial
277905,6041099,604109900034,O,1960-05-24,NaN,NaN,NaN,NaN,04120,NaN,NaN,NaN,NaN,2009-01-01,both,https://data.inpi.fr/entreprises/006041099,initial
6087,6041099,604109900018,O,1960-05-24,NaN,NaN,NaN,NaN,04120,NaN,NaN,NaN,NaN,2018-05-17,both,https://data.inpi.fr/entreprises/006041099,partiel
6088,6041099,604109900026,O,1960-05-24,NaN,NaN,NaN,NaN,04120,NaN,NaN,NaN,NaN,2018-05-17,both,https://data.inpi.fr/entreprises/006041099,partiel
6089,6041099,604109900034,O,1960-05-24,NaN,NaN,NaN,NaN,04120,NaN,NaN,NaN,NaN,2018-05-17,both,https://data.inpi.fr/entreprises/006041099,partiel
